In [1]:
import pandas as pd
import numpy as np
from typing import Any

In [2]:
def write(file: str, data: Any):
    with open(file, 'w') as f:
        f.write(data if isinstance(data, str) else str(data))

def rescale(x: np.ndarray) -> np.ndarray:
    min_ = np.min(x)
    range_ = np.ptp(x)
    return (x - min_) / range_

In [3]:
df = pd.read_csv('time_series_covid19_deaths_US_raw.csv')
raw_data = df.to_numpy()

In [4]:
cum_data = np.cumsum(raw_data, axis=0)
data = np.diff(raw_data)

del raw_data

In [5]:
wisc_cal_data = cum_data[(4, 48), :]
q1_str = '\n'.join((','.join(map(str, x)) for x in wisc_cal_data.tolist()))
# write('q1.txt', q1_str)

del cum_data
del q1_str

In [6]:
q2_data = data[(4, 48), :] # 4-> cal, 48 -> wisco
q2_str = '\n'.join((','.join(map(str, x)) for x in q2_data.tolist()))
# write('q2.txt', q2_str)

del q2_data
del q2_str

In [7]:
mean = np.mean(data, axis=1)
mean = rescale(mean)

In [8]:
std = np.std(data, axis=1)
std = rescale(std)

In [9]:
median = np.median(data, axis=1)
median = rescale(median)

In [10]:
ltc = np.array([sum((x[i] - mean[idx]) * (i + 1 - (half_length := ((data.shape[-1] + 1) / 2))) for i in range(len(x))) for idx, x in enumerate(data.tolist())]) / sum((i+1 - half_length) ** 2 for i in range(data.shape[-1]))
ltc = rescale(ltc)

In [11]:
ac = np.array([ sum((x[i] * x[i-1] for i in range(1, len(x)))) / np.sum(np.power(x, 2)) for x in (data - mean.reshape(50, -1)).tolist()])
ac = rescale(ac)

In [12]:
parametric_data = np.empty((50, 5))
parametric_data[:, 0] = mean
parametric_data[:, 1] = std
parametric_data[:, 2] = median
parametric_data[:, 3] = ltc
parametric_data[:, 4] = ac

q4_str = '\n'.join((','.join(map(lambda s: f'{s:.4f}', x)) for x in np.around(parametric_data, 4).tolist()))
# write('q4.txt', q4_str)

In [ ]:
class HierarchicalClustering:
    class _Linkage:
        _linkages: set = {'single', 'complete'}

        @staticmethod
        def _compute_distances(X: np.ndarray) -> np.ndarray:
            pass
        
        @staticmethod
        def _single(X: np.ndarray):
            pass

        @staticmethod
        def _complete(X: np.ndarray):
            pass

    def __init__(self, n_clusters: int = 2, linkage: str = 'single'):
        pass

    def _build():
        pass

    def fit(self, X: np.ndarray):
        self.X = X
        self._build()

    def predict(self, X: np.ndarray):
        pass